# Fixed Effects: Indicator Variables for Groups

One common use of indicator variables are as *fixed effects*. Fixed effects are used when our data as a "nested" structure (we think individual observations belong to groups), and we suspect different things may be happening in each group. 

For example, suppose we have a dataset of student test scores, and students are all grouped into different schools; or perhaps we have data on earnings and gender across US cities. In these examples, individual observations can be thought of as being grouped into schools or cities. 

One option with this kind of data is to just ignore the groups. For example, if we want to know about differences in the academic performance of minority children across the school system, then we might not want to add controls for students' schools because we think that part of way race impacts performance is though sorting of minority students into worse schools. If we added school fixed effects, we'd lose that variation. 

But suppose we were interested in understanding whether school administrators treat minority children differently, and whether this affects academic performance. Principles, for example, may be more likely to suspect Black children than White children. If that were our interest, then what we really want to know about is how race impacts academic performance *among students in the same school*. And that's where fixed effects are useful -- they let us control for group-level effects (like the fact all children in one school might tend to get lower grades) so we can focus on explaining *intra-group* variation (differences among children *at the same school*).  

In this regard, fixed effects are analogous in purpose to hierarchical models, though they are slightly different in implementation (differences between fixed effects and hierarchical models are [discussed here](fixed_effects_v_hierarchical.ipynb)). 

## Implementing Fixed Effects

To illustrate, let's try and estimate how gender impacts earnings in the US using data from the US Current Population Survey (CPS) on US wages in 2019. We'll begin with a simple model of earnings:

In [1]:
import pandas as pd

# Load survey
cps = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/blob/"
    "master/Current_Population_Survey/morg18.dta?raw=true"
)

# Limit to people currently employed and working full time.
cps = cps[cps.lfsr94 == "Employed-At Work"]
cps = cps[cps.uhourse >= 35]

# Annual earnings from weekly
cps["annual_earnings"] = cps["earnwke"] * 48

# And create gender and college educ variable
cps["female"] = (cps.sex == 2).astype("int")
cps["has_college_educ"] = (cps.grade92 > 43).astype("int")

In [2]:
import statsmodels.formula.api as smf

smf.ols("annual_earnings ~ female + age + has_college_educ", cps).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     8393.
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        10:46:28   Log-Likelihood:            -1.4365e+06
No. Observations:              122603   AIC:                         2.873e+06
Df Residuals:                  122599   BIC:                         2.873e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         3.639e+04    296.455    122.766      0.000    3.58e+04     3.7e+04
female           -1.168e+04    170.390    -68.540      0.000    -1.2e+04   -1.13e+04
age                407.7144      6.406     63.650      0.000     395.160     420.269
has_college_educ  3.054e+04    239.482    127.513      0.000    3.01e+04     3.1e+04
==============================================================================
Omnibus:                    18030.472   Durbin-Watson:                   1.805
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27634.350
Skew:                           1.059   Prob(JB):                         0.00
Kurtosis:                       3.963   Cond. No.                         159.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In this model, we're getting estimates of how education and gender explain variation across all Americans. 

But in this dataset, we also have a variable that tells us the industry in which each respondent is employed. If we want to understand the relationship between gender and income through *both* workplace bias and sectoral sorting, we can use the model above. But suppose we want to estimate wage discrimination in the workplace after controlling for the industry into which someone chooses to work. In other words, we want to know about the impact of gender on wages *within industries*. 

To do so, we can add an indicator for each respondent's industry (in the `ind02` variable):

Then we can run the following regression:

In [3]:
# Adding semi-colon to suppress output because too long...
smf.ols(
    "annual_earnings ~ female + age + has_college_educ + C(ind02)", cps
).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     161.1
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        10:46:31   Log-Likelihood:            -1.4299e+06
No. Observations:              122603   AIC:                         2.860e+06
Df Residuals:                  122341   BIC:                         2.863e+06
Df Model:                         261                                         
Covariance Type:            nonrobust                                         
============================================================================================================================================================================================
                                                                                                                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                  2.27e+04   1156.364     19.629      0.000    2.04e+04     2.5e+04
C(ind02)[T.Animal production (112)]                                                                                       -667.2345   1766.161     -0.378      0.706   -4128.882    2794.413
C(ind02)[T.Forestry except logging (1131, 1132)]                                                                          1.249e+04   3861.556      3.235      0.001    4922.899    2.01e+04
C(ind02)[T.Logging (1133)]                                                                                                6521.2131   3112.166      2.095      0.036     421.420    1.26e+04
C(ind02)[T.Fishing, hunting, and trapping (114)]                                                                          6743.2068   4700.078      1.435      0.151   -2468.867     1.6e+04
C(ind02)[T.Support activities for agriculture and forestry (115)]                                                         6223.7146   2864.905      2.172      0.030     608.548    1.18e+04
C(ind02)[T.Oil and gas extraction (211)]                                                                                  4.164e+04   3187.133     13.064      0.000    3.54e+04    4.79e+04
C(ind02)[T.Coal mining (2121)]                                                                                            2.771e+04   2645.991     10.471      0.000    2.25e+04    3.29e+04
C(ind02)[T.Metal ore mining (2122)]                                                                                       3.934e+04   4341.889      9.061      0.000    3.08e+04    4.79e+04
C(ind02)[T.Nonmetallic mineral mining and quarrying (2123)]                                                               1.895e+04   2480.216      7.641      0.000    1.41e+04    2.38e+04
C(ind02)[T.Support activities for mining (213)]                                                                           3.216e+04   1567.913     20.511      0.000    2.91e+04    3.52e+04
C(ind02)[T.Electric power generation, transmission and distribution (Pt. 2211)]                                           3.006e+04   1487.014     20.214      0.000    2.71e+04     3.3e+04
C(ind02)[T.Natural gas distribution (Pt.s2212)]                                                                           2.286e+04   2571.605      8.891      0.000    1.78e+04    2.79e+04
C(ind02)[T.Electric and gas, and other combinations (Pts. 2

which will generate output that will look approximately like this (note your output will be VERY long -- I'm omitting all the county coeffients for space. We'll talk later about how to suppress those in your output):

![cps_fe_unclustered_p1](images/cps_fe_unclustered_p1.png)

```
               .
               .
               .
```
![cps_fe_unclustered_p2](images/cps_fe_unclustered_p2.png)

Voila! What you've just estimated is no longer the relationship between gender and income across all Americans, but rather the relationship between gender and income *within each industry*. 

To be clear, fixed effects aren't *mathematically* different from adding a normal control variable. One could say that adding `has_college_educ` means that we're now estimating the relationship between gender and income among college educated and among non-college educated. *Mechnically*, fixed effects are just additional indicator variables. But because we often use them for groups, thinking about the fact that, when added, one is effectively estimating variation *within* the groups specified by the fixed effects is a powerful idea. 

Perhaps no place is this more clear than in full panel data, where you have data on the same entities over time. In a panel regression, the addition of entity fixed effects allow you to difference out any *constant* differences between entities, and focus only on changes within each entity over time. This even works for people! In a panel with individuals observed over time, adding individual fixed effects means you're effectively controlling for anything constant about each individual (things that don't change over time), and now you're just studying *changes over time* for each individual. 

## Clustering

When working with fixed effects, however, it's also often a good idea to cluster your standard errors by your fixed effect variable. Clustering is a method for taking into account some of the variation in your data isn't coming from the individual level (where you have lots of observations), but rather from the group level. Since you have fewer groups than observations, clustering corrects your standard errors to reflect the smaller effective sample size being used to estimate those fixed effects (clustering *only* affects standard errors -- it has no impact on coefficients themselves. This is just about adjustments to our confidence in our inferences). 

Clustering is thankfully easy to do—just use the `get_robustcov_results` method from `statsmodels`, and use the `groups` keyword to pass the group assignments for each observation. 

(R users: as we'll discuss below, I think the easiest way to do this is to use the [plm package](https://cran.r-project.org/web/packages/plm/vignettes/plmPackage.html).) 

**TWO IMPORTANT IMPLEMENTATION NOTES:**


1. First, if you're using formulas in statsmodels, the regression is automatically dropping observations that can't be estimated because of missing data, so you have to do the same before passing your group assignments to-`get_robustcov_results`—otherwise you'll get the error:

```
ValueError: The weights and list don't have the same length.
```

because the number of observations in the model doesn't match the number of observations in the group assignment vector you pass!

2. Whatever you pass to `groups` has to be a numeric array of group identifiers. If you don't, you'll get an error like:

```
TypeError: '<' not supported between instances of 'float' and 'str'
```


In [4]:
model = smf.ols(
    "annual_earnings ~ female + age + has_college_educ + C(ind02)", cps
).fit()

# Drop any entries with missing data from the model
fe_groups = cps.copy()
for i in ["annual_earnings", "female", "age", "ind02", "has_college_educ"]:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]

# Convert `ind02` categorical into group codes by
# pulling codes used in its categorical encoding.

# If you have a string instead of a categorical,
# just make it a categorical first with `pd.Categorical()`
group_codes = fe_groups.ind02.cat.codes
group_codes.value_counts()

17     8244
208    8017
220    6716
233    4706
209    3100
       ... 
28       14
31       11
29       10
36        9
186       6
Length: 259, dtype: int64

In [5]:
# Adjust SEs
# Again, suppressing actual output for space with semicolon
model.get_robustcov_results(cov_type="cluster", groups=group_codes).summary()

/Users/nce8/opt/miniconda3/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 262, but rank is 3
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     1144.
Date:                Thu, 16 Feb 2023   Prob (F-statistic):          1.16e-148
Time:                        10:46:33   Log-Likelihood:            -1.4299e+06
No. Observations:              122603   AIC:                         2.860e+06
Df Residuals:                  122341   BIC:                         2.863e+06
Df Model:                         261                                         
Covariance Type:              cluster                                         
============================================================================================================================================================================================
                                                                                                                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                  2.27e+04    707.825     32.067      0.000    2.13e+04    2.41e+04
C(ind02)[T.Animal production (112)]                                                                                       -667.2345     21.509    -31.021      0.000    -709.591    -624.878
C(ind02)[T.Forestry except logging (1131, 1132)]                                                                          1.249e+04    189.349     65.971      0.000    1.21e+04    1.29e+04
C(ind02)[T.Logging (1133)]                                                                                                6521.2131    117.869     55.326      0.000    6289.105    6753.321
C(ind02)[T.Fishing, hunting, and trapping (114)]                                                                          6743.2068     88.305     76.363      0.000    6569.317    6917.097
C(ind02)[T.Support activities for agriculture and forestry (115)]                                                         6223.7146     56.986    109.214      0.000    6111.497    6335.932
C(ind02)[T.Oil and gas extraction (211)]                                                                                  4.164e+04    124.533    334.350      0.000    4.14e+04    4.19e+04
C(ind02)[T.Coal mining (2121)]                                                                                            2.771e+04    100.803    274.861      0.000    2.75e+04    2.79e+04
C(ind02)[T.Metal ore mining (2122)]                                                                                       3.934e+04    109.244    360.142      0.000    3.91e+04    3.96e+04
C(ind02)[T.Nonmetallic mineral mining and quarrying (2123)]                                                               1.895e+04     93.188    203.379      0.000    1.88e+04    1.91e+04
C(ind02)[T.Support activities for mining (213)]                                                                           3.216e+04     76.606    419.797      0.000     3.2e+04    3.23e+04
C(ind02)[T.Electric power generation, transmission and distribution (Pt. 2211)]                                           3.006e+04    140.869    213.383      0.000    2.98e+04    3.03e+04
C(ind02)[T.Natural gas distribution (Pt.s2212)]                                                                           2.286e+04     67.088    340.810      0.000    2.27e+04     2.3e+04
C(ind02)[T.Electric and gas, and other combinations (Pts. 2

![cps_fe_clustered_p1](images/cps_fe_clustered_p1.png)
```
               .
               .
               .
```
![cps_fe_clustered_p2](images/cps_fe_clustered_p2.png)

As you can see, while our point estimates haven't changed at all (the coefficient on `female`, for example, is still -6,892), we have increased the size of our standard errors. The SE on `female`, for example, has gone from 190 without clustering to 420 with clustering.

## Computationally Efficient Fixed Effects

OK, so everything we've describe up till here is a reasonable approach to fixed effects, but it has two limitations: our regression output looks *terrible*, and computing all those intercepts was slow.

This brings us to some of the specialized methods for calculating fixed effects. It turns out that if you aren't interested in the coefficient on each fixed effect, there are much more computationally efficient methods of calculating fixed effects. But to use them, we'll have to use a different library: [linearmodels](https://bashtage.github.io/linearmodels/doc/index.html) (installable using `conda install linearmodels` or `pip install linearmodels`).

(R users: see note at bottom on doing this in R)

In particular, we'll be using the `PanelOLS` function from `linearmodels`. As the name implies, `PanelOLS` is designed for linear regression (social scientists call linear regression Ordinary Least Squares, or OLS) with panel data, which is really any form of data organized along two dimensions. Normally a panel has data on many entities observed several times, so the first dimension is the `entity` dimension, and the second is the `time` dimension. 

In this case, we don't really have a panel—just nested data—but because fixed effects are commonly used in panels, we'll use this tool. 

The only catch is: you have to use `multiindexes` in `pandas`. I *know*, I hate them too. But the multi-index is required by the library for it to understand what variable constitutes the "group" for which you want to add fixed effects. Basically `PanelOLS` calls the first level of the multi-index the `entity` and the second level `time`. In this case, though, we'll just make the first level our counties, and the second level individual identifiers, then use `entity` fixed effects (and clustering).

In [6]:
cps.head()


,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02,annual_earnings,female,has_college_educ
2,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,0,0,1,0,12.0,Office and administrative support occupations,"Health care services , except hospitals",43344.0,1,0
3,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,0,0,0,0,12.0,Office and administrative support occupations,Administrative and support services,19200.0,1,0
4,000110359424339,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0711,07111,1,1,...,0,0,0,0,NaN,Healthcare practitioner and technical occupations,Hospitals,60000.0,1,0
6,000110651278174,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,0,0,0,0,12.0,Transportation and material moving occupations,Transportation and warehousing,32640.0,0,0
17,007680515071194,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0611,06112,2,2,...,0,0,0,0,12.0,Transportation and material moving occupations,Retail trade,38400.0,0,0


In [7]:
# Move county groups into highest level of multi-index,
# with old index in second level.
# PanelOLS will then see the first level as the `entity`
# identifier.
cps_w_multiindex = cps.set_index(["ind02", cps.index])
cps_w_multiindex.head()

,,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02,annual_earnings,female,has_college_educ
ind02,,,,,,,,,,,,,,,,,,,,,,
"Residential care facilities, without nursing (6232, 6233, 6239)",2,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,0,0,1,0,12.0,Office and administrative support occupations,"Health care services , except hospitals",43344.0,1,0
Business support services (5614),3,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,0,0,0,0,12.0,Office and administrative support occupations,Administrative and support services,19200.0,1,0
Hospitals (622),4,000110359424339,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0711,07111,1,1,...,0,0,0,0,NaN,Healthcare practitioner and technical occupations,Hospitals,60000.0,1,0
Truck transportation (484),6,000110651278174,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,0,0,0,0,12.0,Transportation and material moving occupations,Transportation and warehousing,32640.0,0,0
****Department stores and discount stores (s45211),17,007680515071194,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0611,06112,2,2,...,0,0,0,0,12.0,Transportation and material moving occupations,Retail trade,38400.0,0,0


In [9]:
from linearmodels import PanelOLS

mod = PanelOLS.from_formula(
    "annual_earnings ~ 1 + female + age + has_college_educ + EntityEffects",
    data=cps_w_multiindex,
)
mod.fit(cov_type="clustered", cluster_entity=True)

/Users/nce8/opt/miniconda3/lib/python3.10/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,annual_earnings,R-squared:,0.1414
Estimator:,PanelOLS,R-squared (Between):,0.2800
No. Observations:,122603,R-squared (Within):,0.1414
Date:,"Thu, Feb 16 2023",R-squared (Overall):,0.1683
Time:,10:47:02,Log-likelihood,-1.43e+06
Cov. Estimator:,Clustered,,
,,F-statistic:,6716.0
Entities:,259,P-value,0.0000
Avg Obs:,473.37,Distribution:,"F(3,122341)"
Min Obs:,6.0000,,
Max Obs:,8244.0,F-statistic (robust):,303.79


As you can see, the coefficients of the `PanelOLS` model are exactly the same as those we calculated above, and the standard errors are nearly identical (there are a few ways to calculate clustered standard errors, so they aren't numerically equivalent). But the way `PanelOLS` added fixed effects was much more computationally efficient efficient, and in these situations, we don't usually want to see the coefficients, so they're suppressed. 

## Panel Analysis in R

If you're an R user, the best library I am aware of for these types of analyses is `lfe`, which you can [read about here](https://journal.r-project.org/archive/2013-2/gaure.pdf).

The analogue of the `PanelOLS` regression above with `lfe` would be:

```r

# estimate the fixed effects regression with felm()
library(lfe) #fixed effect model package

## Loading required package: Matrix
library(haven) # load stata package

# load data
cps = read_dta('https://github.com/nickeubank/MIDS_Data/blob/master/Current_Population_Survey/morg18.dta?raw=true')

# create dv
cps$earnhre_dollars = cps$earnhre / 100
cps['annual_earnings'] = cps['earnhre_dollars'] * cps['uhourse'] * 52

# model specification
model <- felm(annual_earnings ~ age  | county | 0 |county, data = cps)

# model summary
summary(model)

## 
## Call:
##    felm(formula = annual_earnings ~ age | county | 0 | county, data = cps) 
## 
## Residuals:
##    Min     1Q Median     3Q    Max 
## -63187 -14735  -3622   9682 330934 
## 
## Coefficients:
##     Estimate Cluster s.e. t value Pr(>|t|)    
## age  296.703        5.347   55.49   <2e-16 ***
## ---
## Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
## 
## Residual standard error: 24510 on 93599 degrees of freedom
##   (208632 observations deleted due to missingness)
## Multiple R-squared(full model): 0.03591   Adjusted R-squared: 0.03488 
## Multiple R-squared(proj model): 0.03234   Adjusted R-squared: 0.0313 
## F-statistic(full model, *iid*):34.87 on 100 and 93599 DF, p-value: < 2.2e-16 
## F-statistic(proj model):  3079 on 1 and 99 DF, p-value: < 2.2e-16
```

Where `felm` is the panel regression. 

The syntax for `felm` is:

```
felm(dependent variable ~ independent variable | fixed effects variables (if you have 2+, add them like 'fe1 + fe2')  | instrumental variables (0 if no instrument) | level of clustered se, data )
```

We haven't covered (and probably won't cover) instrumental variables, so you'll usually leave the middle entry as `0`:

```
felm(dependent variable ~ independent variables | fixed effects variables | 0 | level of clustered se, data )
```
